# 2. Evaluación de Sistemas RAG con LangSmith

## Objetivos de Aprendizaje
- Comprender la importancia de la evaluación en sistemas RAG.
- Configurar LangSmith para trazabilidad y evaluación.
- Crear un dataset de evaluación con preguntas y respuestas de referencia.
- Ejecutar evaluadores automáticos para métricas como relevancia y fidelidad.
- Analizar los resultados de la evaluación para optimizar el sistema.

## ¿Qué es LangSmith?

LangSmith es una plataforma de LangChain para la observabilidad, el monitoreo y la evaluación de aplicaciones construidas con Modelos de Lenguaje Grandes (LLMs). Permite visualizar cada paso de una cadena o agente, analizar su rendimiento y evaluar la calidad de las respuestas de forma sistemática.

Para un sistema RAG, LangSmith nos ayuda a responder preguntas clave:
- **Recuperación (Retrieval)**: ¿Los documentos que encontramos son relevantes para la pregunta?
- **Generación (Generation)**: ¿La respuesta generada es fiel a los documentos recuperados? ¿Responde correctamente a la pregunta del usuario?

## 1. Instalación y Configuración

In [ ]:
!pip install --upgrade openai langsmith langchain httpx -q

In [14]:
import os
from openai import OpenAI
from langsmith import Client
import json

print("✅ Librerías importadas")

✅ Librerías importadas


### Configuración de Variables de Entorno

Para que LangSmith capture las trazas de nuestra aplicación, necesitamos configurar cuatro variables de entorno:
1. `LANGCHAIN_TRACING_V2`: Se establece en `"true"` para activar la trazabilidad.
2. `LANGCHAIN_API_KEY`: Tu clave de API de LangSmith. La puedes obtener en [smith.langchain.com](https://smith.langchain.com).
3. `LANGCHAIN_PROJECT`: El nombre del proyecto bajo el cual se agruparán las trazas. Esto es muy útil para organizar el trabajo.
4. `OPENAI_API_KEY`: Tu clave de API de OpenAI para que el modelo de lenguaje funcione.

In [15]:
# Inicializar el cliente de LangSmith
client = Client()

print("✅ Variables de entorno y cliente de LangSmith configurados")

✅ Variables de entorno y cliente de LangSmith configurados


## 2. Sistema RAG Básico

Reutilizaremos el sistema RAG simple del notebook anterior. Este sistema utiliza una lista de documentos en memoria, una función de recuperación por palabras clave y un LLM para generar respuestas.

In [16]:
# Base de documentos
documents = [
    "La inteligencia artificial es una rama de la informática que busca crear máquinas capaces de realizar tareas que requieren inteligencia humana.",
    "Los modelos de lenguaje grande (LLM) son sistemas de IA entrenados en enormes cantidades de texto para generar y comprender lenguaje natural.",
    "RAG (Retrieval-Augmented Generation) combina la búsqueda de información relevante con la generación de texto para producir respuestas más precisas.",
    "LangChain es un framework que facilita el desarrollo de aplicaciones con modelos de lenguaje, proporcionando herramientas para cadenas y agentes.",
    "El prompt engineering es la práctica de diseñar instrucciones efectivas para obtener los mejores resultados de los modelos de IA."
]

# Cliente de OpenAI (siguiendo el patrón del proyecto)
def initialize_client():
    client = OpenAI(
        base_url=os.getenv("GITHUB_BASE_URL", "https://models.inference.ai.azure.com"),
        api_key=os.getenv("GITHUB_TOKEN")
    )
    return client

openai_client = initialize_client()

print(f"📚 Base de datos con {len(documents)} documentos cargada.")
print("✅ Cliente OpenAI inicializado correctamente.")

📚 Base de datos con 5 documentos cargada.
✅ Cliente OpenAI inicializado correctamente.


In [17]:
from langsmith.run_helpers import traceable

# Envolvemos las funciones con el decorador @traceable para que LangSmith las capture

@traceable(name="Recuperacion de Documentos")
def simple_retrieval(query, documents):
    relevant_docs = []
    query_lower = query.lower()
    for doc in documents:
        if any(word in doc.lower() for word in query_lower.split()):
            relevant_docs.append(doc)
    return relevant_docs[:3]

@traceable(name="Generacion de Respuesta")
def generate_response(client, query, context):
    prompt = f"""Contexto:
{context}

Pregunta: {query}

Responde basándote únicamente en el contexto proporcionado."""
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    return response.choices[0].message.content

@traceable(name="Pipeline RAG Completo")
def rag_pipeline(query):
    context_docs = simple_retrieval(query, documents)
    context = "".join(context_docs)
    answer = generate_response(openai_client, query, context)
    return {"answer": answer, "context": context_docs}

print("✅ Funciones del pipeline RAG definidas y trazables")

✅ Funciones del pipeline RAG definidas y trazables


### Prueba de Trazabilidad

Ahora, si ejecutamos nuestro pipeline, LangSmith registrará la ejecución completa, incluyendo los pasos intermedios que decoramos. Puedes ir a tu proyecto en LangSmith para ver la traza.

In [18]:
resultado = rag_pipeline("¿Qué es RAG?")
print(resultado["answer"])

Exception ignored in: <function SyncHttpxClientWrapper.__del__ at 0x113aff4c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/openai/_base_client.py", line 801, in __del__
    if self.is_closed:
       ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/httpx/_client.py", line 228, in is_closed
    return self._state == ClientState.CLOSED
           ^^^^^^^^^^^
AttributeError: 'SyncHttpxClientWrapper' object has no attribute '_state'


RAG (Retrieval-Augmented Generation) es un enfoque que combina la búsqueda de información relevante con la generación de texto para producir respuestas más precisas.


## 3. Creación de un Dataset de Evaluación

Para evaluar nuestro sistema, necesitamos un "ground truth" (verdad fundamental), es decir, un conjunto de preguntas y las respuestas que consideramos correctas. En LangSmith, esto se gestiona a través de Datasets.

In [19]:
dataset_name = "Dataset RAG Básico - Fundamentos IA"
description = "Preguntas y respuestas sobre conceptos básicos de IA para evaluar un RAG simple."

# Eliminar dataset si ya existe para evitar duplicados
try:
    existing_dataset = client.read_dataset(dataset_name=dataset_name)
    client.delete_dataset(dataset_id=str(existing_dataset.id))
    print(f"🗑️ Dataset '{dataset_name}' existente eliminado.")
except Exception:
    pass # El dataset no existía, no hay nada que hacer

# Crear el nuevo dataset
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description=description,
)

print(f"✅ Dataset '{dataset_name}' creado.")

🗑️ Dataset 'Dataset RAG Básico - Fundamentos IA' existente eliminado.
✅ Dataset 'Dataset RAG Básico - Fundamentos IA' creado.


### Añadir Ejemplos al Dataset

Cada ejemplo consta de:
- `inputs`: Un diccionario con las entradas de nuestro sistema (en este caso, la `query`).
- `outputs`: Un diccionario con la salida de referencia que esperamos (la `answer` correcta).

In [20]:
client.create_example(
    inputs={"query": "¿Qué es la inteligencia artificial?"},
    outputs={"answer": "La inteligencia artificial es una rama de la informática que busca crear máquinas capaces de realizar tareas que requieren inteligencia humana."},
    dataset_id=dataset.id,
)

client.create_example(
    inputs={"query": "¿Para qué sirve LangChain?"},
    outputs={"answer": "LangChain es un framework que facilita el desarrollo de aplicaciones con modelos de lenguaje, proporcionando herramientas para cadenas y agentes."},
    dataset_id=dataset.id,
)

client.create_example(
    inputs={"query": "Explica qué es RAG"},
    outputs={"answer": "RAG (Retrieval-Augmented Generation) combina la búsqueda de información relevante con la generación de texto para producir respuestas más precisas."},
    dataset_id=dataset.id,
)

print(f"✅ 3 ejemplos añadidos al dataset '{dataset_name}'.")

✅ 3 ejemplos añadidos al dataset 'Dataset RAG Básico - Fundamentos IA'.


## 4. Ejecución de la Evaluación

Ahora que tenemos el sistema y el dataset, podemos ejecutar la evaluación. LangSmith utilizará un LLM para comparar las respuestas generadas por nuestro `rag_pipeline` con las respuestas de referencia (`ground truth`) de nuestro dataset.

In [21]:
from langsmith.evaluation import evaluate
from langchain_openai import ChatOpenAI
from langchain.evaluation import load_evaluator

# Cargamos un evaluador predefinido de LangChain. "qa" es un evaluador de exactitud.
evaluator = load_evaluator("qa")

# La función que será evaluada. Debe aceptar 'inputs' como un diccionario.
def target_function(inputs):
    return rag_pipeline(inputs["query"])

experiment_results = evaluate(
    target_function,  # La función a evaluar
    data=dataset_name,  # El nombre de nuestro dataset en LangSmith
    evaluators=[evaluator], # Pasamos la lista de objetos evaluadores
    experiment_prefix="RAG Básico - Primera Prueba", # Prefijo para el nombre del experimento
    metadata={"version": "1.0.0"}, # Metadatos para seguimiento
)

print("✅ Evaluación completada. Revisa los resultados en LangSmith.")

View the evaluation results for experiment: 'RAG Básico - Primera Prueba-844ffd37' at:
https://smith.langchain.com/o/3a147d9f-375a-49e5-b37f-01cf8bf3ea9d/datasets/0c2e95e5-a5a5-4133-812c-1f5f54a9a810/compare?selectedSessions=8c0b5725-7396-4e11-99f6-9b355281e432




0it [00:00, ?it/s]Exception ignored in: Exception ignored in: <function SyncHttpxClientWrapper.__del__ at 0x113aff4c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/openai/_base_client.py", line 801, in __del__
<function SyncHttpxClientWrapper.__del__ at 0x113aff4c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/openai/_base_client.py", line 801, in __del__
Exception ignored in: <function SyncHttpxClientWrapper.__del__ at 0x113aff4c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/openai/_base_client.py", line 801, in __del__
    if self.is_closed:
       ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/httpx/_client.py", line 228, in is_closed
    if self.is_closed:
       ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/httpx/_client.py", line 228, in is_closed
    if self.is_closed:
       ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-pack

✅ Evaluación completada. Revisa los resultados en LangSmith.


## 5. Análisis de Resultados

Una vez completada la evaluación, puedes navegar a la pestaña **Experiments** en tu proyecto de LangSmith.

Allí encontrarás:
1. Un resumen del experimento con las puntuaciones medias de cada métrica.
2. Una tabla detallada con cada pregunta del dataset, la respuesta generada, la respuesta de referencia y las puntuaciones de la evaluación.
3. Para cada fila, puedes hacer clic para ver la traza completa y entender por qué el sistema respondió de esa manera (qué documentos recuperó, qué prompt se usó, etc.).

Este análisis te permite identificar puntos débiles. Por ejemplo:
- **Puntuaciones bajas de `correctness`**: Puede que la recuperación no esté funcionando bien o que el prompt de generación necesite ajustes.
- **Documentos irrelevantes en el contexto**: Indica que el método de `retrieval` debe ser mejorado (por ejemplo, pasando de búsqueda por palabras clave a búsqueda semántica con embeddings).

## Conclusión

La evaluación es un pilar fundamental en el desarrollo de sistemas de IA robustos. LangSmith nos ofrece un conjunto de herramientas poderosas para automatizar este proceso en sistemas RAG, permitiéndonos pasar de un desarrollo basado en la intuición a uno guiado por datos y métricas objetivas. Con este enfoque, podemos mejorar de forma iterativa la calidad y fiabilidad de nuestras aplicaciones.